In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#gap = pd.read_csv('c:\DatasRev\gapminder.csv')
gap = pd.read_csv('https://raw.githubusercontent.com/DatasRev/workshop-prep/master/06_data_cleaning_2/gapminder.csv')
gap.head()

Life expentancy oszlopot indexnek illetve Country-ra átnevezni, NaN értékek kezelése, Unnamed oszlopot eltűntetni,tidy data három oszlopra (country, year, life_expectancy).

In [ ]:
# Adatszett vizsgálata, 780 sor, 219 oszlop, amiből egy object, a többi float és int.
gap.info()

In [ ]:
# Az object típusú oszlop jelen esetben string, nincsenek közte számok, nem kell típuskonverzió.
# 780 sor van, több, mint a jelenlegi országok száma.

gap.select_dtypes(include=['object']).head()

Dropolni kell az Unnamed: 0 oszlopot, anélkül nem fogja tudni jól értelmezni a pd.melt parancsot. Meg kell adni az axis-t is, jelen esetben az 1-et, az jelenti az oszlopot, anélkül KeyError: "['Unnamed: 0'] not found in axis" hibát fog dobni.

In [ ]:
gap = gap.drop('Unnamed: 0', 1)
gap

Táblázat összeolvassztása, a 'Life expectancy' oszlopot indexként használva. Átnevezzük az oszlopokat az áttekinthetőség miatt, hogy ne variable, meg value legyen a két név.

In [ ]:
gap_melt = pd.melt(gap, id_vars='Life expectancy')
gap_melt.columns = ['country', 'year', 'life_expectancy']
gap_melt.head()

In [ ]:
# A melt után a year oszlop object, azaz string lett, ezt vissza kell alakítani.
gap_melt.info()

Az assert paranccsal ellenőrizhető az átalakítás eredménye. Ha nem ad vissza hibaüzenetet, az azt jelenti, hogy átment a teszten, TRUE-ra értékelődött. Mivel itt kevés az oszlop, a könnyebb láthatóság kedvéért lekértem a .info()-t is, int64 lett.

In [ ]:
gap_melt['year'] = pd.to_numeric(gap_melt['year'])
assert gap_melt.year.dtypes == np.int64
gap_melt.info()

A következő lépés a 'country' oszlop tartalmának átnézése, van-e bármilyen speciális, vagy helytelen karakter a nevekben. Ehhez először szűrjük az oszlopot a duplikációtól. 

In [ ]:
countries = gap_melt['country']
countries.count()

In [ ]:
countries = countries.drop_duplicates()
countries.count()

Regexp segítségével összeállítunk egy mintát, hogy szerintünk mik szerepelnek egy országnévben: kisbetű, nagybetű, pont, szóköz.
"Anchor the pattern to match exactly what you want by placing a ^ in the beginning and $ in the end.
Use A-Za-z to match the set of lower and upper case letters, \. to match periods, and \s to match whitespace between words."

A mask változónak odaadjuk ezt a mintát, kihasználva a countries stringeknél használható contains parancsot, ami bool értéket ad vissza.

A mask-ot meghívva látjuk, hogy egy series az eredmény, True/False értékekkel.

A ~ karakterrel megfordítjuk az eredményt, ez látszik is a példánál.

In [ ]:
pattern = '^[A-Za-z\.\s]*$'
mask = countries.str.contains(pattern)
mask.head()

In [ ]:
mask_inverse = ~mask
mask_inverse.head()

Lekérjük azokat az országokat, amikben általunk nem engedett karakterek vannak.

In [ ]:
invalid_countries = countries.loc[mask_inverse]
invalid_countries

Ellenőrizzük, hogy a year, és a country oszlopokban nincs-e NULL sor. A life_expentancyt nem kell ellenőrizni, arról tudjuk, hogy rengeteg a hiányos adat. A dropna után negyedelődik a sorok száma.

In [ ]:
gap_melt.count()

In [ ]:
assert pd.notnull(gap_melt.country).all()
assert pd.notnull(gap_melt.year).all()
gap_melt = gap_melt.dropna()
gap_melt.count()

In [ ]:
gap_melt.sample(10)

Most, hogy megvan a tisztított adatszett, egy gyors vizualizációval megnézhetjük, hogy jónak tűnnek-e a life_expentancy adatok. Ha nincs 0-nál kisebb adat, az jó jel.

In [ ]:
gap_melt['life_expectancy'].plot(kind='hist', bins=15)

A végső vizualizációval megnézzük, hogyan változott az évek alatt az átlagéletkor. Ehhez groupby-olni kell évenként az átlagéletkort, és megjeleníteni.

In [ ]:
gap_agg = gap_melt.groupby('year')['life_expectancy'].mean()
gap_agg.tail()

In [ ]:
plt.title('Life expectancy over the years')
plt.ylabel('Life expectancy')
plt.xlabel('Year')
gap_agg.plot()

Ha már ennyit dolgoztunk, illik el is menteni.

In [ ]:
gap_melt.to_csv('gapminder.csv')
gap_agg.to_csv('gapminder_agg.csv')